# DataClassic examples

In [7]:
from dataclassic.doc_store import DocumentStore, Database
from dataclassic.dataclasses import dataclass, field, DataClassicValidationError, to_json, from_json

In [8]:
# The validator gets the instance of the class
def RealShapeValidator(shape: "Shape"):    
    "A shape must have at least three sides"
    return shape.sides > 2


@dataclass
class Shape:
    ID: str = field(converter=str)  # we can add an extra convert argument here
    sides: int = field(converter=int, validator=RealShapeValidator)  # we can also add a validator
    color: str = field(converter=str)

In [9]:
hexagon = Shape(ID="hexagon", sides=6, color="green")

In [10]:
# This will fail the RealShapeValidator
duogon = Shape(ID="duogon", sides=2, color="maroon")

DataClassicValidationError: Validation for Shape.sides failed.

# Serialization to JSON

This isn't any big deal, as regular dataclasses make this pretty easy

In [11]:
s = to_json(hexagon)
print(f"{s=}")
hexagon2 = from_json(s, Shape)
print(f"{hexagon2=}")

s='{"ID": "hexagon", "sides": 6, "color": "green"}'
hexagon2=Shape(ID='hexagon', sides=6, color='green')


# Type Coercion

Here we make the side a string value.  The type coercion automatically turns it into a float.
Regular dataclasses don't do this.

In [12]:
text = '{"ID": "hexagaon", "sides": "6", "color": "green"}'  # made 6 a string value
hexagon3 = from_json(text, Shape)
assert hexagon3.sides == 6
print(hexagon3)

Shape(ID='hexagaon', sides=6, color='green')


# Document Store

In [13]:
# Define a bunch of shapes that we will want to store
hexagon = Shape(ID="hexagon", sides=6, color="green")
triangle = Shape(ID="triangle", sides="3", color="red")  # type coercion will work here too.
rectangle = Shape(ID="rectangle", sides=4, color="blue")
pentagon = Shape(ID="pentagon", sides=5, color="red")
print(f"{triangle=}")

triangle=Shape(ID='triangle', sides=3, color='red')


In [14]:
# We'll define a sqlite database (in memory for this example) and a shapes document store
db = Database("sqlite:///:memory:")
shapes = DocumentStore("shapes", db, False, dtype=Shape)

In [15]:
# We can insert them all at once
shapes.insert_many((triangle, rectangle, pentagon, hexagon))

[{'ID': 'triangle', 'sides': 3, 'color': 'red'},
 {'ID': 'rectangle', 'sides': 4, 'color': 'blue'},
 {'ID': 'pentagon', 'sides': 5, 'color': 'red'},
 {'ID': 'hexagon', 'sides': 6, 'color': 'green'}]

In [16]:
# Data is stored in a two column table - ID and Document are the columns
# This bit is just to show how the data is stored.  Next we can
# use the shapes collection to more easily do queries
cursor = db.cursor()
cursor.execute("select * from collectionj_shapes")
{item["ID"]:item["Document"] for item in cursor.fetchall()}

{'triangle': '{"ID": "triangle", "sides": 3, "color": "red"}',
 'rectangle': '{"ID": "rectangle", "sides": 4, "color": "blue"}',
 'pentagon': '{"ID": "pentagon", "sides": 5, "color": "red"}',
 'hexagon': '{"ID": "hexagon", "sides": 6, "color": "green"}'}

# Queries

    =======             ======================
    OP                  Equivalent SQL
    =======             ======================
    $and, and           and
    $in                 in
    $or, or             or
    &like, like         like
    $nin, not in        not in
    $between, between   between
    $null, null         is null
    $nnull, not null    is not null
    $gt, gt, >          >
    $gte, gte, >=       >=
    $lt, lt, <          <
    $lte, lte, <=       <=
    $ne, ne, <>, !=     <>
    $eq, eq, =, ==      =


In [17]:
# Get all of the shapes
shapes.find()

[Shape(ID='triangle', sides=3, color='red'),
 Shape(ID='rectangle', sides=4, color='blue'),
 Shape(ID='pentagon', sides=5, color='red'),
 Shape(ID='hexagon', sides=6, color='green')]

In [18]:
# We can query like this:
shapes.find("@sides > 4")

[Shape(ID='pentagon', sides=5, color='red'),
 Shape(ID='hexagon', sides=6, color='green')]

In [19]:
# And we can query like this:
shapes.find("@sides > 3 and @sides < 6")

[Shape(ID='rectangle', sides=4, color='blue'),
 Shape(ID='pentagon', sides=5, color='red')]

In [20]:
# Query to find things in or not in a set
not_blue_red = shapes.find("@color not in ('blue', 'red')")
blue_red = shapes.find("@color in ('blue', 'red')")

print(f"{not_blue_red=}\n{blue_red=}")

not_blue_red=[Shape(ID='hexagon', sides=6, color='green')]
blue_red=[Shape(ID='triangle', sides=3, color='red'), Shape(ID='rectangle', sides=4, color='blue'), Shape(ID='pentagon', sides=5, color='red')]


In [21]:
# A MongoDB like syntax is also supported
print(shapes.find2({"$eq": {"@color" : 'blue'}}))
print(shapes.find2({"$in": {"@color" : ('blue', 'red')}}))

[Shape(ID='rectangle', sides=4, color='blue')]
[Shape(ID='triangle', sides=3, color='red'), Shape(ID='rectangle', sides=4, color='blue'), Shape(ID='pentagon', sides=5, color='red')]


In [22]:
data = shapes.find()

    


In [23]:
data

[Shape(ID='triangle', sides=3, color='red'),
 Shape(ID='rectangle', sides=4, color='blue'),
 Shape(ID='pentagon', sides=5, color='red'),
 Shape(ID='hexagon', sides=6, color='green')]

In [27]:
colmap = dict(enumerate(Shape.__dataclass_fields__.keys()))

In [30]:
def getd(i, j):

    return getattr(data[i], colmap[j])

In [31]:
getd(1,1)

4

In [34]:
from dataclassic.dctables import DataClassTable


In [35]:
t=DataClassTable(data)

In [36]:
t

AttributeError: 'DataClassTable' object has no attribute 'name'